## Create CNN Model And Optimize it Using Keras Tuner

In [32]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras

In [33]:
print(tf.__version__)

2.14.0


In [34]:
fashion_mnist = keras.datasets.fashion_mnist

In [35]:
(train_images,train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [36]:
train_images = train_images/255.0
test_images = test_images/255.0

In [37]:
train_images[0].shape

(28, 28)

In [38]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [39]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [40]:
from keras_tuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [41]:
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    directory='output',
    project_name="Mnist_Fashion"
)

Reloading Tuner from output/Mnist_Fashion/tuner0.json


In [42]:
tuner.search(train_images, train_labels, epochs=3, validation_split=0.1)

In [43]:
model = tuner.get_best_models(num_models=1)[0]

In [44]:
model.summary

<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x7e4a285f99f0>>

In [45]:
model.fit(train_images,train_labels,epochs=10, validation_split=0.1 , initial_epoch=2)

Epoch 3/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1866 - accuracy: 0.9307 - val_loss: 0.2864 - val_accuracy: 0.9038
Epoch 4/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1515 - accuracy: 0.9426 - val_loss: 0.2812 - val_accuracy: 0.9003
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1235 - accuracy: 0.9530 - val_loss: 0.2930 - val_accuracy: 0.9095
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1013 - accuracy: 0.9620 - val_loss: 0.3051 - val_accuracy: 0.9100
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0804 - accuracy: 0.9690 - val_loss: 0.3526 - val_accuracy: 0.9112
Epoch 8/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0675 - accuracy: 0.9749 - val_loss: 0.3613 - val_accuracy: 0.9128
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0568 - accuracy: 0.9789 - val_loss: 0.4053 - val_accurac